In [10]:
from fastai.vision import *
from fastai.metrics import error_rate
from pathlib import Path

from sklearn.metrics import confusion_matrix

import pandas as pd
import numpy as np
import os


import re


In [11]:
# get path of the folder to extract for Batching
path = Path(os.getcwd())/"data"
path


PosixPath('/home/ot420/dm/Waste-Sorter/data')

In [12]:
tfms = get_transforms(do_flip=True,flip_vert=True)
data = ImageDataBunch.from_folder(path,test="test",ds_tfms=tfms,bs=16)

In [13]:
data

ImageDataBunch;

Train: LabelList (1262 items)
x: ImageList
Image (3, 384, 512),Image (3, 384, 512),Image (3, 384, 512),Image (3, 384, 512),Image (3, 384, 512)
y: CategoryList
paper,paper,paper,paper,paper
Path: /home/ot420/dm/Waste-Sorter/data;

Valid: LabelList (630 items)
x: ImageList
Image (3, 384, 512),Image (3, 384, 512),Image (3, 384, 512),Image (3, 384, 512),Image (3, 384, 512)
y: CategoryList
paper,paper,paper,paper,paper
Path: /home/ot420/dm/Waste-Sorter/data;

Test: LabelList (635 items)
x: ImageList
Image (3, 384, 512),Image (3, 384, 512),Image (3, 384, 512),Image (3, 384, 512),Image (3, 384, 512)
y: EmptyLabelList
,,,,
Path: /home/ot420/dm/Waste-Sorter/data

In [14]:
#Catagories:
print(data.classes)

['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']


In [15]:
#show a batch
data.show_batch(rows=4,figsize=(10,8))

RuntimeError: CUDA out of memory. Tried to allocate 36.00 MiB (GPU 0; 3.95 GiB total capacity; 140.15 MiB already allocated; 18.75 MiB free; 3.85 MiB cached)

In [ ]:
learn = cnn_learner(data,models.resnet34,metrics=error_rate)

In [ ]:
# show info of the model
learn.model


In [ ]:
learn.lr_find(start_lr=1e-6,end_lr=1e1)
learn.recorder.plot(suggestion=True)

In [ ]:
learn.fit_one_cycle(20,max_lr=3.55e-03)

In [ ]:
preds = learn.get_preds(ds_type=DatasetType.Test)

Don't pass in data,pass in the dataset type. passing data gives an error.

In [ ]:
print(preds[0].shape)
preds[0]

These are the predicted probablities for each image. This tensor has 365 rows,one for each image and 6 column,one for each material category.

In [ ]:
data.classes

convert probabilites to their corresponding class

In [ ]:

max_idxs = np.asarray(np.argmax(preds[0],axis=1))

In [ ]:
ypred = []
for max_idx in max_idxs:
    ypred.append(data.classes[max_idx])

In [ ]:
ypred

These are the predicted labels of all the images.

In [ ]:
learn.data.test_ds[0][0]
#picture of first img

In [ ]:
y = []

#convert file paths to string 
for label_path in data.test_ds.items:
    y.append(str(label_path))

#extract waste type from file path
pattern = re.compile("([a-z]+)[0-9]+")
for i in range(len(y)):
    s = y[i]
    s = s[len(s)//2:]
    y[i] = s
    y[i] = pattern.search(y[i]).group(1)


In [ ]:
cm = confusion_matrix(y,ypred)
correct = 0

for r in range(len(cm)):
    for c in range(len(cm)):
        if (r==c):
            correct += cm[r,c]


accuracy = correct/sum(sum(cm))
print("The accuracy of the model is:",accuracy*100)



In [ ]:
print("Actual: ",y[50:60])
print("Predicted: ",ypred[50:60])

